In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate
import copy
from datetime import datetime


print("Import Success")

Import Success


In [2]:
meta = pd.read_csv('Dataset/movies_metadata.csv')
meta.head()

/usr/local/Caskroom/miniconda/base/envs/recommender/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
# Rating
ratings = pd.read_csv('Dataset/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
#Links of IMDb and TMDb
links = pd.read_csv('Dataset/links_small.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
keywords = pd.read_csv('Dataset/keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [6]:
# Content based Recommender System

meta['overview'] = meta['overview'].fillna('')
meta['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [7]:
pd.DataFrame({'feature':meta.dtypes.index, 'dtype':meta.dtypes.values})

,feature,dtype
0,adult,object
1,belongs_to_collection,object
2,budget,object
3,genres,object
4,homepage,object
5,id,object
6,imdb_id,object
7,original_language,object
8,original_title,object
9,overview,object


In [8]:
meta = meta.drop([19730, 29503, 35587]) # Remove these ids to solve ValueError: "Unable to parse string..."

meta['id'] = pd.to_numeric(meta['id'])

In [9]:
pd.DataFrame({'feature':links.dtypes.index, 'dtype':links.dtypes.values})

,feature,dtype
0,movieId,int64
1,imdbId,int64
2,tmdbId,float64


In [10]:
col=np.array(links['tmdbId'], np.int64)
links['tmdbId']=col

In [11]:
meta.rename(columns={'id':'tmdbId'}, inplace=True)
meta = pd.merge(meta,links,on='tmdbId')
meta.drop(['imdb_id'], axis=1, inplace=True)
meta.head()


,adult,belongs_to_collection,budget,genres,homepage,tmdbId,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1,114709
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,2,113497
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,3,113228
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,4,114885
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5,113041


In [12]:
tfidf = TfidfVectorizer(stop_words='english')


# Constructing  matrix TF-IDF
tfidf_matrix = tfidf.fit_transform(meta['overview'])
tfidf_matrix.shape

(9099, 29727)

In [13]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(meta.index, index=meta['original_title']).drop_duplicates()

In [14]:
def recommend(title, cosine_sim=cosine_sim):

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:16]
    movie_indices = [i[0] for i in sim_scores]
    for i in movie_indices:
        pop = meta.at[i,'vote_average']
        if pop<5 or pop>10:
            movie_indices.remove(i)

    return meta[['original_title','vote_average']].iloc[movie_indices]

In [15]:
recommend('Iron Man')

,original_title,vote_average
7516,Iron Man 2,6.6
8296,Iron Man 3,6.8
5670,Scarface,7.5
8766,Avengers: Age of Ultron,7.3
8100,Brake,5.3
4274,Saturday Night Fever,6.5
6063,Hostage,6.2
2322,The Dark Half,5.4
6152,Batman Begins,7.5
1650,Return from Witch Mountain,5.6


In [16]:

reader = Reader() 
df = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(df) 

<generator object KFold.split at 0x7fe0bbf08c80>

In [17]:
svd = SVD()
cross_validate(svd, df, measures=['RMSE', 'MAE'])

trainset = df.build_full_trainset()
svd.fit(trainset)

In [18]:
ratings[ratings['userId'] == 10]

,userId,movieId,rating,timestamp
744,10,50,5.0,942766420
745,10,152,4.0,942766793
746,10,318,4.0,942766515
747,10,344,3.0,942766603
748,10,345,4.0,942766603
749,10,592,3.0,942767328
750,10,735,4.0,942766974
751,10,1036,3.0,942767258
752,10,1089,3.0,942766420
753,10,1101,2.0,942767328


In [19]:
# smaller link file reload 
links_df = pd.read_csv('Dataset/links_small.csv')
col=np.array(links_df['tmdbId'], np.int64)
links_df['tmdbId']=col

links_df = links_df.merge(meta[['title', 'tmdbId']], on='tmdbId').set_index('title')
links_index = links_df.set_index('tmdbId') 

In [20]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = links_df.loc[title]['tmdbId'] # Get the corresponding tmdb id
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] # Scores of 30 similar movies
    movie_indices = [i[0] for i in sim_scores]
    
    movies = meta.iloc[movie_indices][['title', 'vote_average', 'tmdbId']]
    movies['est'] = movies['tmdbId'].apply(lambda x: svd.predict(userId, links_index.loc[x]['movieId']).est) # Estimated prediction using svd
    movies = movies.sort_values('est', ascending=False) # Rank movies according to the predicted values
    movies.columns = ['Title', 'Vote Average', 'TMDb Id', 'Estimated Prediction']
    return movies.head(30) # Display top 30 recommended movies

In [28]:
hybrid(30,'The Conjuring')

result = hybrid(10,'Batman Begins')
print("data getting passed in contextual")
print(result)

data getting passed in contextual
                                        Title  Vote Average  TMDb Id  \
6909                          The Dark Knight           8.3      155   
8238  Batman: The Dark Knight Returns, Part 2           7.9   142061   
6937                                 Iron Man           7.4     1726   
2581             Batman: Mask of the Phantasm           7.4    14919   
7284          Gangster's Paradise: Jerusalema           6.8    22600   
8144                        Seven Psychopaths           6.7    86838   
7941                    The Dark Knight Rises           7.6    49026   
4524                     Bang the Drum Slowly           6.3    42459   
8175  Batman: The Dark Knight Returns, Part 1           7.7   123025   
7911                         Batman: Year One           7.1    69735   
9063                               I Am Wrath           4.5   332411   
7392                                   Shrink           6.1    22640   
7575               Batman: Und

In [29]:
# necessary functions for contextual_update function

def day_time():
    now = datetime.now().time()
    morning=now.replace(hour=12,minute=0,second=0,microsecond=0)
    afternoon=now.replace(hour=16,minute=0,second=0,microsecond=0)
    evening=now.replace(hour=19,minute=0,second=0,microsecond=0)

    if now< morning :
        return "morning"
    elif now<afternoon :
        return "afternoon"
    elif now<evening :
        return "evening"
    else :
        return "night"

def season():
    month = datetime.now().month

    if month < 4:
        return "winter"
    elif month <6:
        return "summer"
    elif month <9:
        return "rainy"
    elif month < 11:
        return "autumn"
    else :
        return  "winter"

def is_weekend():
    day=datetime.now().isoweekday()
    if day< 6:
        return False
    return True


#testing function
#day_time()
season()

'winter'

In [30]:
# Function to include movies on specific dates -
def special_date(recommended_list,date_passed):
    print("special date function reached")
    date_event = datetime.now().date()

    # Independence Day
    date_event=date_event.replace(month=8,day=15)
    new_list=recommended_list.copy()
    if date_event == date_passed:
        # Vote Average  TMDb Id  Estimated Prediction
        new_movie = pd.DataFrame({"Title":["Border","Uri:The Surgical Strike"],
                                  "Vote Average":[6.8,7.1],
                                  "TMDb Id":[33125,554600],
                                  "Estimated Prediction":[5.0,5.0],
                                  "tmdbId":[33125,554600],
                                  "genres":["[{'name':'Action'},{'name':'History'},{'name':'War'}]","[{'name':'Action'},{'name':'Drama'},{'name':'War'}]"]
                                  })
        new_list = pd.concat([new_movie,recommended_list])
    #Repubic Day
    date_event=date_event.replace(month=1,day=26)
    if date_event == date_passed:
        new_movie = pd.DataFrame({"Title":["Shaheed","Border","Uri:The Surgical Strike"],
                                  "Vote Average":[5.0,6.8,7.1],
                                  "TMDb Id":[498713,33125,554600],
                                  "Estimated Prediction":[5.0,5.0,5.0],
                                  "tmdbId":[498713,33125,554600],
                                  "genres":["[{'name':'War'},{'name':'History'}]","[{'name':'Action'},{'name':'History'},{'name:'War'}]","[{'name':'Action'},{'name':'Drama'},{'name':'War'}]"]
                                  })
        new_list = pd.concat([new_movie,recommended_list])
    #Teachers Day
    date_event=date_event.replace(month=9,day=5)
    if date_event == date_passed:
        new_movie = pd.DataFrame({"Title":["Super 30","Taare Zameen Par"],
                                  "Vote Average":[7.6,8.0],
                                  "TMDb Id":[534075,7508],
                                  "Estimated Prediction":[5.0,5.0],
                                  "tmdbId":[534075,7508],
                                  "genres":["[{'name':'Drama'}]","[{'name':'Drama'}]"]
                                  })
        new_list = pd.concat([new_movie,recommended_list])
    #Children day
    date_event=date_event.replace(month=11,day=14)
    if date_event == date_passed:
        new_movie = pd.DataFrame({"Title":["Taare Zameen Par","Chillar Party"],
                                  "Vote Average":[8.0,6.9],
                                  "TMDb Id":[7508,69891],
                                  "Estimated Prediction":[5.0,5.0],
                                  "tmdbId":[7508,69891],
                                  "genres":["[{'name':'Drama'}]","[{'name':'Drama'},{'name':'Comedy'},{'name':'Family'}]"]
                                  })
        new_list = pd.concat([new_movie,recommended_list])
    #Christmas
    date_event=date_event.replace(month=12,day=25)
    if date_event == date_passed:
        new_movie = pd.DataFrame({"Title":["Let It Snow","Home Alone"],
                                  "Vote Average":[6.1,7.3],
                                  "TMDb Id":[295151,771],
                                  "Estimated Prediction":[5.0,5.0],
                                  "tmdbId":[295151,771],
                                  "genres":["[{'name':'Romance'},{'name':'Comedy'}]","[{'name':'Comedy'},{'name':'Family'}]"]
                                  })
        new_list = pd.concat([new_movie,recommended_list])
    #New Year
    date_event=date_event.replace(month=12,day=31)
    if date_event == date_passed:
        new_movie = pd.DataFrame({"Title":["New Years Eve"],
                                  "Vote Average":[5.9],
                                  "TMDb Id":[62838],
                                  "Estimated Prediction":[5.0],
                                  "tmdbId":[62838],
                                  "genres":["[{'name':'Comedy'},{'name':'Romance'}]"]
                                  })
        new_list = pd.concat([new_movie,recommended_list])
    date_event=date_event.replace(month=1,day=1)
    if date_event == date_passed:
        new_movie = pd.DataFrame({"Title":["New Years Eve"],
                                  "Vote Average":[5.9],
                                  "TMDb Id":[62838],
                                  "Estimated Prediction":[5.0],
                                  "tmdbId":[62838],
                                  "genres":["[{'name':'Comedy'},{'name':'Romance'}]"]
                                  })
        new_list = pd.concat([new_movie,recommended_list])
    #Valentine
    date_event=date_event.replace(month=2,day=14)
    if date_event == date_passed:
        new_movie = pd.DataFrame({"Title":["The Notebook","Titanic"],
                                  "Vote Average":[7.9,7.9],
                                  "TMDb Id":[11036,597],
                                  "Estimated Prediction":[5.0,5.0],
                                  "tmdbId":[11036,597],
                                  "genres":["[{'name':'Romance'},{'name':'Drama'}]","[{'name':'Drama'},{'name':'Romance'}]"]
                                  })
        new_list = pd.concat([new_movie,recommended_list])
    
    return new_list


In [31]:
def recommendation_updater(recommended_list,genre_score):
    #print("reached recommendation updater - ")
    new_list=recommended_list.copy()
    for ind in recommended_list.index:
        new_score=0
        movie_genre= list(eval(recommended_list['genres'][ind]))
        #print(recommended_list['genres'][ind])
        #print(type(recommended_list['genres'][ind]))
        #print(movie_genre)
        curr_genre_list= [li['name'] for li in movie_genre]
        #print(curr_genre_list)
        for genre in curr_genre_list:
            if genre in genre_score:
                new_score+=genre_score[genre]
        #print(new_score)
        new_list['Estimated Prediction'][ind]=new_list['Estimated Prediction'][ind]+new_score
    return new_list

In [32]:

def contextual_update(list_passed,family=False,device="Mobile",no_of_people=1,date_passed=datetime.now().date()) :
    # categories we have romance,action,comedy,drama ,crime and thriller ,documentary,sci-fi
    recommended_list=list_passed.copy()
    print("Before Context-Awareness based changes - ")
    print(list_passed)

    # Adding Genres for update
    recommended_list = pd.merge(recommended_list,meta[['tmdbId','genres']],left_on=['TMDb Id'],right_on=['tmdbId']).dropna()

    # Special Days
    test_date=datetime.now().date()
    test_date=test_date.replace(month=8,day=15)
    recommended_list=special_date(recommended_list,test_date)
    recommended_list.reset_index(drop=True,inplace=True)

    # Reducing score to take account for contextual_update

    effect_rate = 0.75
    category=4
    recommended_list['Estimated Prediction']=recommended_list['Estimated Prediction']-effect_rate


    # Timing based

    day_part = day_time()
    if day_part == "morning":
        scores={
        'Romance':0.24*(effect_rate/category),'Action':0.18*(effect_rate/category),'Comedy':0.64*(effect_rate/category),'Drama':0.24*(effect_rate/category),'Crime':0.17*(effect_rate/category)
        ,'Thriller':0.17*(effect_rate/category),'Documentary':0.25*(effect_rate/category),'Science Fiction':0.28*(effect_rate/category)
        }
    elif day_part =="afternoon":
        scores ={
           'Romance':0.18*(effect_rate/category),'Action':0.44*(effect_rate/category),'Comedy':0.48*(effect_rate/category),'Drama':0.35*(effect_rate/category),'Crime':0.5*(effect_rate/category)
        ,'Thriller':0.5*(effect_rate/category),'Documentary':0.24*(effect_rate/category),'Science Fiction':0.35*(effect_rate/category)
        }
    elif day_part =="evening":
        scores={
        'Romance':0.4*(effect_rate/category),'Action':0.34*(effect_rate/category),'Comedy':0.48*(effect_rate/category),'Drama':0.3*(effect_rate/category),'Crime':0.4*(effect_rate/category)
        ,'Thriller':0.4*(effect_rate/category),'Documentary':0.24*(effect_rate/category),'Science Fiction':0.32*(effect_rate/category)
        }
    else :
        scores={
        'Romance':0.57*(effect_rate/category),'Action':0.37*(effect_rate/category),'Comedy':0.42*(effect_rate/category),'Drama':0.37*(effect_rate/category),'Crime':0.54*(effect_rate/category)
        ,'Thriller':0.54*(effect_rate/category),'Documentary':0.31*(effect_rate/category),'Science Fiction':0.41*(effect_rate/category)
        }
    recommended_list=recommendation_updater(recommended_list,scores)

    # Season based
    curr_season = season()
    if curr_season == "summer":
        scores={
        'Romance':0.32*(effect_rate/category),'Action':0.48*(effect_rate/category),'Comedy':0.57*(effect_rate/category),'Drama':0.5*(effect_rate/category),'Crime':0.6*(effect_rate/category)
        ,'Thriller':0.6*(effect_rate/category),'Documentary':0.27*(effect_rate/category),'Science Fiction':0.47*(effect_rate/category)
        }
    elif curr_season == "rainy":
        scores={
        'Romance':0.57*(effect_rate/category),'Action':0.3*(effect_rate/category),'Comedy':0.52*(effect_rate/category),'Drama':0.5*(effect_rate/category),'Crime':0.41*(effect_rate/category)
        ,'Thriller':0.41*(effect_rate/category),'Documentary':0.14*(effect_rate/category),'Science Fiction':0.32*(effect_rate/category)
        }
    elif curr_season == "autumn":
        scores={
        'Romance':0.41*(effect_rate/category),'Action':0.37*(effect_rate/category),'Comedy':0.5*(effect_rate/category),'Drama':0.48*(effect_rate/category),'Crime':0.52*(effect_rate/category)
        ,'Thriller':0.52*(effect_rate/category),'Documentary':0.31*(effect_rate/category),'Science Fiction':0.44*(effect_rate/category)
        }
    else :
        scores={
        'Romance':0.54*(effect_rate/category),'Action':0.45*(effect_rate/category),'Comedy':0.51*(effect_rate/category),'Drama':0.42*(effect_rate/category),'Crime':0.5*(effect_rate/category)
        ,'Thriller':0.5*(effect_rate/category),'Documentary':0.21*(effect_rate/category),'Science Fiction':0.32*(effect_rate/category)
        }
    recommended_list=recommendation_updater(recommended_list,scores)

    # Weekday based -

    if is_weekend():
        scores={
        'Romance':0.41*(effect_rate/category),'Action':0.48*(effect_rate/category),'Comedy':0.54*(effect_rate/category),'Drama':0.38*(effect_rate/category),'Crime':0.7*(effect_rate/category)
        ,'Thriller':0.7*(effect_rate/category),'Documentary':0.28*(effect_rate/category),'Science Fiction':0.41*(effect_rate/category)
        }
    else :
        scores={
        'Romance':0.37*(effect_rate/category),'Action':0.32*(effect_rate/category),'Comedy':0.51*(effect_rate/category),'Drama':0.32*(effect_rate/category),'Crime':0.48*(effect_rate/category)
        ,'Thriller':0.48*(effect_rate/category),'Documentary':0.21*(effect_rate/category),'Science Fiction':0.38*(effect_rate/category)
        }

    recommended_list=recommendation_updater(recommended_list,scores)

    # Device Based

    if  device == "phone":
        scores={
        'Romance':0.36*(effect_rate/category),'Action':0.24*(effect_rate/category),'Comedy':0.66*(effect_rate/category),'Drama':0.44*(effect_rate/category),'Crime':0.38*(effect_rate/category)
        ,'Thriller':0.38*(effect_rate/category),'Documentary':0.2*(effect_rate/category),'Science Fiction':0.21*(effect_rate/category)
        }
    elif device =="tablet":
        scores={
        'Romance':0.34*(effect_rate/category),'Action':0.37*(effect_rate/category),'Comedy':0.43*(effect_rate/category),'Drama':0.43*(effect_rate/category),'Crime':0.42*(effect_rate/category)
        ,'Thriller':0.42*(effect_rate/category),'Documentary':0.22*(effect_rate/category),'Science Fiction':0.36*(effect_rate/category)
        }
    else :
        scores={
        'Romance':0.33*(effect_rate/category),'Action':0.6*(effect_rate/category),'Comedy':0.24*(effect_rate/category),'Drama':0.3*(effect_rate/category),'Crime':0.66*(effect_rate/category)
        ,'Thriller':0.66*(effect_rate/category),'Documentary':0.21*(effect_rate/category),'Science Fiction':0.58*(effect_rate/category)
        }
    recommended_list=recommendation_updater(recommended_list,scores)

    # Based on Number of people and Family -

    if no_of_people >1 :
        if family:
            scores={
            'Romance':0.1*(effect_rate/category),'Action':0.43*(effect_rate/category),'Comedy':0.66*(effect_rate/category),'Drama':0.49*(effect_rate/category),'Crime':0.26*(effect_rate/category)
        ,'Thriller':0.26*(effect_rate/category),'Documentary':0.36*(effect_rate/category),'Science Fiction':0.29*(effect_rate/category)
        }
        else :
            scores={
        'Romance':0.33*(effect_rate/category),'Action':0.63*(effect_rate/category),'Comedy':0.54*(effect_rate/category),'Drama':0.33*(effect_rate/category),'Crime':0.61*(effect_rate/category)
        ,'Thriller':0.61*(effect_rate/category),'Documentary':0.17*(effect_rate/category),'Science Fiction':0.54*(effect_rate/category)
        }

        recommended_list=recommendation_updater(recommended_list,scores)

    # removing genre from table

    recommended_list.drop(['tmdbId','genres'],axis=1,inplace=True)

    # Sorting the list for final result and comparing
    #print(list_passed)
    recommended_list.sort_values(by='Estimated Prediction',ascending=False,inplace=True)
    print(recommended_list)

contextual_update(result)


Before Context-Awareness based changes - 
                                        Title  Vote Average  TMDb Id  \
6909                          The Dark Knight           8.3      155   
8238  Batman: The Dark Knight Returns, Part 2           7.9   142061   
6937                                 Iron Man           7.4     1726   
2581             Batman: Mask of the Phantasm           7.4    14919   
7284          Gangster's Paradise: Jerusalema           6.8    22600   
8144                        Seven Psychopaths           6.7    86838   
7941                    The Dark Knight Rises           7.6    49026   
4524                     Bang the Drum Slowly           6.3    42459   
8175  Batman: The Dark Knight Returns, Part 1           7.7   123025   
7911                         Batman: Year One           7.1    69735   
9063                               I Am Wrath           4.5   332411   
7392                                   Shrink           6.1    22640   
7575               Bat

<ipython-input-31-912d3540313e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_list['Estimated Prediction'][ind]=new_list['Estimated Prediction'][ind]+new_score
